## 问题定义

本问题为求解三维Taylor Green涡流。


### 求解目标

假定雷诺数为500，要求求解边长为$2\pi$，边界条件为周期边界条件并且具有如下初始条件的Taylor Green涡流。

![IC](./resource/IC_eq.png)

## Moving Time Window

移动窗口法本质上是通过将长时间拆解为多个短时序进行顺序求解从而缓解PINN在长时序求解中的不足。

示意图如下：

![Moving Time Window](./resource/MTW.png)

对于每一个Time Window，前面的求解结果会被作为新的初始条件监督后面的训练，每一个Window也是重新初始化的（存在误差累积的问题）


## 求解

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["PYTORCH_JIT"]="0"  # 需要手动禁用JIT，否则MovingTimeWindowArch会报错

In [2]:
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra.utils import compose

import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np

from sympy import Symbol, Eq, Abs, sin, cos

import modulus.sym
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.utils.io import csv_to_dict
from modulus.sym.solver import SequentialSolver
from modulus.sym.domain import Domain

from modulus.sym.geometry.primitives_3d import Box

from modulus.sym.models.fully_connected import FullyConnectedArch
from modulus.sym.models.moving_time_window import MovingTimeWindowArch
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from modulus.sym.domain.inferencer import PointVTKInferencer
from modulus.sym.utils.io import (
    VTKUniformGrid,
)
from modulus.sym.key import Key
from modulus.sym.node import Node
from modulus.sym.eq.pdes.navier_stokes import NavierStokes

In [3]:
cfg = compose(config_path="conf", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 300000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: ${training.rec_results_freq}
  rec_inference_freq: ${training.rec_results_freq}
  rec_monitor_freq: ${training.rec_results_freq}
  rec_constraint_freq: 5000
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: false
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: false
run_mode: train
arc

### 定义必要组件

#### PDE

In [5]:
ns = NavierStokes(nu=0.002, rho=1.0, dim=3, time=True)

#### Time Window Params

In [6]:
time_window_size = 1.0  # 窗口宽度
t_symbol = Symbol("t")
time_range = {t_symbol: (0, time_window_size)}
nr_time_windows = 10  # 窗口数量

#### Geo

In [7]:
# define sympy variables to parametrize domain curves
x, y, z = Symbol("x"), Symbol("y"), Symbol("z")

# make geometry for problem
channel_length = (0.0, 2 * np.pi)
channel_width = (0.0, 2 * np.pi)
channel_height = (0.0, 2 * np.pi)
box_bounds = {x: channel_length, y: channel_width, z: channel_height}

# define geometry
rec = Box(
    (channel_length[0], channel_width[0], channel_height[0]),
    (channel_length[1], channel_width[1], channel_height[1]),
)

#### Model

In [8]:

# 定义简单的全连接网络
# 输入为空间坐标和时间
# 输出为三个方向速度以及压强
flow_net = FullyConnectedArch(
    input_keys=[Key("x"), Key("y"), Key("z"), Key("t")],
    output_keys=[Key("u"), Key("v"), Key("w"), Key("p")],
    periodicity={"x": channel_length, "y": channel_width, "z": channel_height},  # 周期边界，允许模型进行周期性预测
    layer_size=256,
)
time_window_net = MovingTimeWindowArch(flow_net, time_window_size)  # 定义MTW

#### Node

In [9]:
nodes = ns.make_nodes() + [time_window_net.make_node(name="time_window_network")]

#### Domain

在Domain中定义约束以及训练所需的各种组件

在本例中分别声明IC的Domain和MTW的Domain。原因在于对于不同的Window，IC是一直变化的

In [10]:
# make initial condition domain
ic_domain = Domain("initial_conditions")

# make moving window domain
window_domain = Domain("window")

#### 初始条件

In [11]:
# make initial condition
# 真实初始条件，第一个Window
ic = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={
        "u": sin(x) * cos(y) * cos(z),
        "v": -cos(x) * sin(y) * cos(z),
        "w": 0,
        "p": 1.0 / 16 * (cos(2 * x) + cos(2 * y)) * (cos(2 * z) + 2),
    },
    batch_size=cfg.batch_size.initial_condition,
    bounds=box_bounds,
    lambda_weighting={"u": 100, "v": 100, "w": 100, "p": 100},
    parameterization={t_symbol: 0},
)
ic_domain.add_constraint(ic, name="ic")


# make constraint for matching previous windows initial condition
# 对于后续每一个Window，其初始条件是上一个Window预测的结果
ic = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"u_prev_step_diff": 0, "v_prev_step_diff": 0, "w_prev_step_diff": 0},
    batch_size=cfg.batch_size.interior,
    bounds=box_bounds,
    lambda_weighting={
        "u_prev_step_diff": 100,
        "v_prev_step_diff": 100,
        "w_prev_step_diff": 100,
    },
    parameterization={t_symbol: 0},  # 时间上也重置为0
)
window_domain.add_constraint(ic, name="ic")

#### PDE约束

In [12]:
# make interior constraint
interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"continuity": 0, "momentum_x": 0, "momentum_y": 0, "momentum_z": 0},
    bounds=box_bounds,
    batch_size=4094,
    parameterization=time_range,
)
ic_domain.add_constraint(interior, name="interior")
window_domain.add_constraint(interior, name="interior")

验证器以及其他必要组件

In [13]:
for i, specific_time in enumerate(np.linspace(0, time_window_size, 10)):  # 对于每一个切片均运行结果
    vtk_obj = VTKUniformGrid(
        bounds=[(0, 2 * np.pi), (0, 2 * np.pi), (0, 2 * np.pi)],
        npoints=[128, 128, 128],
        export_map={"u": ["u", "v", "w"], "p": ["p"]},
    )
    
    # 创建Inferencer
    grid_inference = PointVTKInferencer(
        vtk_obj=vtk_obj,
        nodes=nodes,
        input_vtk_map={"x": "x", "y": "y", "z": "z"},
        output_names=["u", "v", "w", "p"],
        requires_grad=False,
        invar={"t": np.full([128**3, 1], specific_time)},  # t序列
        batch_size=100000,
    )
    ic_domain.add_inferencer(grid_inference, name="time_slice_" + str(i).zfill(4))
    window_domain.add_inferencer(
        grid_inference, name="time_slice_" + str(i).zfill(4)
    )

### 求解器以及求解

In [14]:
# 定义求解器
# 这里使用序列求解器
# 不同于一般的求解器
# 序列求解器要求传入求解次数-求解域对
slv = SequentialSolver(
    cfg,
    [(1, ic_domain), (nr_time_windows, window_domain)],  # 表示： ic_domain运行一次，window_domain运行nr_time_windows次
    custom_update_operation=time_window_net.move_window,  # 表示更新参数
)

手动加载日志系统

In [15]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [16]:
slv.solve()

2024-02-28 02:47:58,849 - INFO - Solving for Domain initial_conditions, iteration 0
2024-02-28 02:47:59,220 - INFO - attempting to restore from: /workspace/12_3D_TaylorGreenFlow_MovingTimeWindow/outputs/initial_conditions
2024-02-28 02:47:59,222 - WARNING - optimizer checkpoint not found
2024-02-28 02:47:59,223 - WARNING - model time_window_network.0.pth not found
2024-02-28 02:48:00,756 - INFO - [step:          0] record constraint batch time:  2.318e-01s
2024-02-28 02:48:31,199 - INFO - [step:          0] record inferencers time:  3.044e+01s
2024-02-28 02:48:31,216 - INFO - [step:          0] saved checkpoint to /workspace/12_3D_TaylorGreenFlow_MovingTimeWindow/outputs/initial_conditions
2024-02-28 02:48:31,218 - INFO - [step:          0] loss:  6.627e+03
2024-02-28 02:48:35,542 - INFO - Attempting cuda graph building, this may take a bit...
2024-02-28 02:48:50,528 - INFO - [step:        100] loss:  5.723e+01, time/iteration:  1.931e+02 ms
2024-02-28 02:48:55,615 - INFO - [step:     

2024-02-28 02:57:14,727 - INFO - [step:       6000] loss:  4.842e+00, time/iteration:  3.822e+02 ms
2024-02-28 02:57:20,463 - INFO - [step:       6100] loss:  3.652e+00, time/iteration:  5.735e+01 ms
2024-02-28 02:57:25,839 - INFO - [step:       6200] loss:  4.118e+00, time/iteration:  5.374e+01 ms
2024-02-28 02:57:31,216 - INFO - [step:       6300] loss:  2.993e+00, time/iteration:  5.376e+01 ms
2024-02-28 02:57:36,593 - INFO - [step:       6400] loss:  3.602e+00, time/iteration:  5.375e+01 ms
2024-02-28 02:57:41,968 - INFO - [step:       6500] loss:  3.021e+00, time/iteration:  5.373e+01 ms
2024-02-28 02:57:47,342 - INFO - [step:       6600] loss:  3.434e+00, time/iteration:  5.373e+01 ms
2024-02-28 02:57:52,719 - INFO - [step:       6700] loss:  4.879e+00, time/iteration:  5.375e+01 ms
2024-02-28 02:57:58,095 - INFO - [step:       6800] loss:  4.417e+00, time/iteration:  5.374e+01 ms
2024-02-28 02:58:03,471 - INFO - [step:       6900] loss:  2.712e+00, time/iteration:  5.376e+01 ms


2024-02-28 03:06:23,448 - INFO - [step:      12700] loss:  1.079e+00, time/iteration:  5.368e+01 ms
2024-02-28 03:06:28,816 - INFO - [step:      12800] loss:  1.679e+00, time/iteration:  5.366e+01 ms
2024-02-28 03:06:34,180 - INFO - [step:      12900] loss:  4.576e-01, time/iteration:  5.363e+01 ms
2024-02-28 03:07:11,245 - INFO - [step:      13000] record inferencers time:  3.071e+01s
2024-02-28 03:07:11,262 - INFO - [step:      13000] saved checkpoint to /workspace/12_3D_TaylorGreenFlow_MovingTimeWindow/outputs/initial_conditions
2024-02-28 03:07:11,264 - INFO - [step:      13000] loss:  2.736e+00, time/iteration:  3.708e+02 ms
2024-02-28 03:07:16,281 - INFO - [step:      13100] loss:  3.308e-01, time/iteration:  5.016e+01 ms
2024-02-28 03:07:21,298 - INFO - [step:      13200] loss:  5.779e-01, time/iteration:  5.016e+01 ms
2024-02-28 03:07:26,262 - INFO - [step:      13300] loss:  4.500e-01, time/iteration:  4.963e+01 ms
2024-02-28 03:07:31,311 - INFO - [step:      13400] loss:  8.6

2024-02-28 03:15:45,883 - INFO - [step:      19200] loss:  1.589e-01, time/iteration:  5.053e+01 ms
2024-02-28 03:15:51,237 - INFO - [step:      19300] loss:  2.105e-01, time/iteration:  5.353e+01 ms
2024-02-28 03:15:56,192 - INFO - [step:      19400] loss:  1.150e-01, time/iteration:  4.953e+01 ms
2024-02-28 03:16:01,156 - INFO - [step:      19500] loss:  3.740e-01, time/iteration:  4.963e+01 ms
2024-02-28 03:16:06,118 - INFO - [step:      19600] loss:  5.803e-01, time/iteration:  4.960e+01 ms
2024-02-28 03:16:11,105 - INFO - [step:      19700] loss:  3.201e-01, time/iteration:  4.985e+01 ms
2024-02-28 03:16:16,066 - INFO - [step:      19800] loss:  5.288e-01, time/iteration:  4.959e+01 ms
2024-02-28 03:16:21,028 - INFO - [step:      19900] loss:  5.944e-01, time/iteration:  4.960e+01 ms
2024-02-28 03:16:27,156 - INFO - [step:      20000] record constraint batch time:  2.330e-01s
2024-02-28 03:16:57,447 - INFO - [step:      20000] record inferencers time:  3.029e+01s
2024-02-28 03:16:

2024-02-28 03:24:32,572 - INFO - [step:      25800] loss:  9.216e-02, time/iteration:  5.050e+01 ms
2024-02-28 03:24:38,320 - INFO - [step:      25900] loss:  3.119e-01, time/iteration:  5.745e+01 ms
2024-02-28 03:25:16,182 - INFO - [step:      26000] record inferencers time:  3.114e+01s
2024-02-28 03:25:16,199 - INFO - [step:      26000] saved checkpoint to /workspace/12_3D_TaylorGreenFlow_MovingTimeWindow/outputs/initial_conditions
2024-02-28 03:25:16,201 - INFO - [step:      26000] loss:  4.788e-01, time/iteration:  3.788e+02 ms
2024-02-28 03:25:21,336 - INFO - [step:      26100] loss:  6.939e-02, time/iteration:  5.134e+01 ms
2024-02-28 03:25:26,452 - INFO - [step:      26200] loss:  1.947e-01, time/iteration:  5.113e+01 ms
2024-02-28 03:25:31,505 - INFO - [step:      26300] loss:  1.556e-01, time/iteration:  5.051e+01 ms
2024-02-28 03:25:36,556 - INFO - [step:      26400] loss:  7.260e-02, time/iteration:  5.050e+01 ms
2024-02-28 03:25:41,513 - INFO - [step:      26500] loss:  2.4

...

2024-02-28 13:45:47,837 - INFO - [step:       8300] loss:  1.001e-01, time/iteration:  7.093e+01 ms
2024-02-28 13:45:57,730 - INFO - [step:       8400] loss:  2.252e-01, time/iteration:  9.891e+01 ms
2024-02-28 13:46:06,556 - INFO - [step:       8500] loss:  5.403e-01, time/iteration:  8.824e+01 ms
2024-02-28 13:46:17,361 - INFO - [step:       8600] loss:  1.685e-01, time/iteration:  1.080e+02 ms
2024-02-28 13:46:29,862 - INFO - [step:       8700] loss:  2.318e-01, time/iteration:  1.250e+02 ms
2024-02-28 13:46:42,313 - INFO - [step:       8800] loss:  2.086e-01, time/iteration:  1.245e+02 ms
2024-02-28 13:46:58,019 - INFO - [step:       8900] loss:  1.556e-01, time/iteration:  1.571e+02 ms
2024-02-28 13:47:56,800 - INFO - [step:       9000] record inferencers time:  4.355e+01s
2024-02-28 13:47:56,820 - INFO - [step:       9000] saved checkpoint to /workspace/12_3D_TaylorGreenFlow_MovingTimeWindow/outputs/window_0000
2024-02-28 13:47:56,822 - INFO - [step:       9000] loss:  3.622e-01,

2024-02-28 14:00:55,682 - INFO - [step:      15000] record inferencers time:  3.916e+01s
2024-02-28 14:00:55,704 - INFO - [step:      15000] saved checkpoint to /workspace/12_3D_TaylorGreenFlow_MovingTimeWindow/outputs/window_0000
2024-02-28 14:00:55,706 - INFO - [step:      15000] loss:  9.122e-02, time/iteration:  4.783e+02 ms
2024-02-28 14:01:02,665 - INFO - [step:      15100] loss:  3.752e-01, time/iteration:  6.959e+01 ms
2024-02-28 14:01:12,158 - INFO - [step:      15200] loss:  1.122e-01, time/iteration:  9.491e+01 ms
2024-02-28 14:01:21,461 - INFO - [step:      15300] loss:  9.018e-02, time/iteration:  9.302e+01 ms
2024-02-28 14:01:30,515 - INFO - [step:      15400] loss:  7.382e-02, time/iteration:  9.053e+01 ms
2024-02-28 14:01:39,585 - INFO - [step:      15500] loss:  3.273e-01, time/iteration:  9.069e+01 ms
2024-02-28 14:01:48,057 - INFO - [step:      15600] loss:  8.890e-02, time/iteration:  8.472e+01 ms
2024-02-28 14:01:57,068 - INFO - [step:      15700] loss:  2.619e-01,

2024-02-28 14:13:28,834 - INFO - [step:      21500] loss:  1.674e-01, time/iteration:  7.970e+01 ms
2024-02-28 14:13:37,571 - INFO - [step:      21600] loss:  2.299e-01, time/iteration:  8.736e+01 ms
2024-02-28 14:13:46,140 - INFO - [step:      21700] loss:  9.342e-02, time/iteration:  8.568e+01 ms
2024-02-28 14:13:55,329 - INFO - [step:      21800] loss:  1.755e-01, time/iteration:  9.189e+01 ms
2024-02-28 14:14:04,669 - INFO - [step:      21900] loss:  4.726e-02, time/iteration:  9.339e+01 ms
2024-02-28 14:14:56,833 - INFO - [step:      22000] record inferencers time:  4.050e+01s
2024-02-28 14:14:56,853 - INFO - [step:      22000] saved checkpoint to /workspace/12_3D_TaylorGreenFlow_MovingTimeWindow/outputs/window_0000
2024-02-28 14:14:56,855 - INFO - [step:      22000] loss:  1.482e-01, time/iteration:  5.218e+02 ms
2024-02-28 14:15:04,052 - INFO - [step:      22100] loss:  4.840e-02, time/iteration:  7.196e+01 ms
2024-02-28 14:15:10,709 - INFO - [step:      22200] loss:  7.216e-02,

2024-02-28 14:27:19,905 - INFO - [step:      28000] loss:  2.178e-01, time/iteration:  5.099e+02 ms
2024-02-28 14:27:26,867 - INFO - [step:      28100] loss:  3.565e-02, time/iteration:  6.960e+01 ms
2024-02-28 14:27:35,177 - INFO - [step:      28200] loss:  1.060e-01, time/iteration:  8.309e+01 ms
2024-02-28 14:27:44,884 - INFO - [step:      28300] loss:  7.123e-02, time/iteration:  9.705e+01 ms
2024-02-28 14:27:54,191 - INFO - [step:      28400] loss:  5.708e-02, time/iteration:  9.305e+01 ms
2024-02-28 14:28:03,491 - INFO - [step:      28500] loss:  1.590e-01, time/iteration:  9.298e+01 ms
2024-02-28 14:28:13,117 - INFO - [step:      28600] loss:  7.336e-02, time/iteration:  9.624e+01 ms
2024-02-28 14:28:22,782 - INFO - [step:      28700] loss:  2.035e-01, time/iteration:  9.663e+01 ms
2024-02-28 14:28:31,817 - INFO - [step:      28800] loss:  7.698e-02, time/iteration:  9.032e+01 ms
2024-02-28 14:28:40,640 - INFO - [step:      28900] loss:  9.464e-02, time/iteration:  8.822e+01 ms


KeyboardInterrupt: 

### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

In [ ]:
import glob
import matplotlib.pyplot as plt

In [ ]:
# get list of steps
window_dirs = glob.glob("./outputs/network_checkpoint/*")
window_dirs.sort()
window_dirs = [x for x in window_dirs if os.path.isdir(x)]

# read each file in each dir and store tke
index = 0
time_points = []
tke_points = []
for i, d in enumerate(window_dirs):
    # get list of slices
    slice_files = glob.glob(d + "/inferencers/time_slice_*.npz")
    slice_files.sort()

    for f in slice_files:
        predicted_data = np.load(f, allow_pickle=True)["arr_0"].item()

        # shift t
        predicted_data["t"] += i
        if float(predicted_data["t"][0, 0, 0]) < 10.0:
            # store time
            time_points.append(float(predicted_data["t"][0, 0, 0]))

            # compute tke and store
            tke = np.mean(
                predicted_data["u"] ** 2 / 2
                + predicted_data["v"] ** 2 / 2
                + predicted_data["w"] ** 2 / 2
            )
            tke_points.append(tke)
            index += 1
tke_points = tke_points / np.max(tke_points)

In [ ]:
# load validation tke data
file_path = "openfoam/validation_tke"
if os.path.exists(to_absolute_path(file_path)):
    validation_tke_128 = csv_to_dict("openfoam/validation_tke/tke_mean_Re500_N128.csv")
    validation_tke_256 = csv_to_dict("openfoam/validation_tke/tke_mean_Re500_N256.csv")

    plt.plot(
        validation_tke_128["Time"][:, 0],
        validation_tke_128["TKE_mean"][:, 0],
        label="Spectral Solver (grid res: 128)",
    )
    plt.plot(
        validation_tke_256["Time"][:, 0],
        validation_tke_256["TKE_mean"][:, 0],
        label="Spectral Solver (grid res: 256)",
    )
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
    )

# plot turbulent kinetic energy decay
plt.plot(time_points, tke_points, label="Modulus")

plt.legend()
plt.title("TKE")
plt.ylabel("TKE")
plt.xlabel("time")
plt.show()